In [1]:
import glob
import os, re 
import numpy as np
import tensorflow as tf

# 1. 데이터 불러오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'    # * 를 이용해 전체 파일을 불러온다.

txt_list = glob.glob(txt_file_path)

raw_corpus = []   # 불러온 텍스트 파일을 담는 리스트

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


# 2. 문장 중에 대화가 있는 문장만 골라내기(Tensorflow의 데이터 전처리)

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0 : continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 2: break   # 문장 3개 확인해보기
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?


In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 5 .. 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

# 처리된 문장 확인하기
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [5]:
# 추출한 문장을 담는 리스트
corpus = []

#  원하는 문장만 추출
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 결과물을 3개 확인해 본다
corpus[:3]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>']

## 토큰화 시작

[Tensorflow](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences)

### maxlen을 이용하면 시퀸스의 최대길이를 정할 수 있다

In [6]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  #12000개의 단어장 
        filters=' ',     # 데이터를 정제 했기때문에 사용할 필요가 없다
        oov_token="<unk>"
    )
    # corpus 함수를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15) 
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
print(tensor[:3, :10])     # 문장 뒤에 padding이 붙어서 0이 출력되는 것을 확인할 수 있다.

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f14612344c0>
[[   2   50    5   91  297   65   57    9  969 6042]
 [   2   17 2639  873    4    8   11 6043    6  329]
 [   2   36    7   37   15  164  282   28  299    4]]


In [7]:
for idx in tokenizer.index_word:       #tokenizer에 구축된 단어 사전의 인덱스
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


### padding이란 데이터의 크기를 맞춰주기 위해 특정 값을 채워넣는 것이다

In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다, padding 값은 의미가 없으므로 제거한다.
rc_input = tensor[:, :-1]   # 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다. 출력값을 보면 끝 부분이 0인 것이 padding
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    # 앞에서부터 진행한다. start를 잘라준다.

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


# 3. 데이터를 훈련데이터와 평가 데이터(전체의 20%)로 분리한다.

In [10]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input, 
                                                          test_size=0.2,
                                                          random_state=42,) 

# 4. 데이터 셋 객체 만들기

In [11]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# 5. 작사 모델 만들기

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256        # 배치 사이즈
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

embedding_size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만, 충분한 데이터가 주어지지 않으면 오히려 혼란만을 야기할 수 있다.
hidden_size 값은 '판단하는 사람의 수'라고 생각할 수 있다. 충분한 데이터가 있으면 상관이 없지만 적은 데이터라면 오히려 혼란을 일으킬 수 있다. 사공이 많으면 배가 산으로 간다는 말이다.

In [15]:
# 데이터셋에서 데이터 한 배치만 불러온다.
for src_sample, tgt_sample in dataset.take(1): break
# 테스트 용으로 한 배치만 불러온 데이터를 모델에 넣어보았다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-8.72402961e-05, -9.28012450e-05, -1.18685632e-04, ...,
          7.11265748e-05,  1.94095162e-04,  1.62847755e-05],
        [-3.44120548e-04, -3.63957457e-04, -2.25052878e-04, ...,
         -1.73488825e-05,  3.31190211e-04,  9.82931397e-06],
        [-7.41630618e-04, -6.60670106e-04, -3.21213913e-04, ...,
         -1.53919376e-04,  5.25018899e-04,  7.05226630e-05],
        ...,
        [-1.72573794e-03, -7.94656051e-04,  1.04067195e-03, ...,
          6.71265938e-04,  1.68912829e-05, -5.36252512e-04],
        [-1.97361223e-03, -1.11946324e-03,  8.09121237e-04, ...,
          5.77226107e-04,  1.73955632e-04, -3.55791504e-04],
        [-2.05686409e-03, -1.04225602e-03,  8.51469464e-04, ...,
          6.49749592e-04,  2.21561902e-04, -2.32820821e-04]],

       [[-7.12693654e-05, -4.00051722e-05, -7.08139341e-05, ...,
          2.85087357e-04, -1.13417031e-04,  8.53467282e-05],
        [-3.01605352e-04,  3.36235353e-05, -1

In [16]:
model.summary()   #모델의 구조 확인

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


# 6. 손실함수 loss 구해보기

## [loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy)

In [17]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 112s 160ms/step - loss: 3.6116
Epoch 2/10
686/686 [==============================] - 111s 162ms/step - loss: 3.1332
Epoch 3/10
686/686 [==============================] - 112s 162ms/step - loss: 2.9400
Epoch 4/10
686/686 [==============================] - 112s 162ms/step - loss: 2.7907
Epoch 5/10
686/686 [==============================] - 112s 162ms/step - loss: 2.6616
Epoch 6/10
686/686 [==============================] - 112s 162ms/step - loss: 2.5448
Epoch 7/10
686/686 [==============================] - 112s 162ms/step - loss: 2.4363
Epoch 8/10
686/686 [==============================] - 112s 162ms/step - loss: 2.3350
Epoch 9/10
686/686 [==============================] - 112s 162ms/step - loss: 2.2383
Epoch 10/10
686/686 [==============================] - 112s 162ms/step - loss: 2.1464


In [19]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [22]:
generate_text(model, tokenizer, init_sentence="<start> she")

'<start> she s got me runnin round and round <end> '